# Predicting Customer Churn with Machine Learning

Beta Bank customers are leaving: little by little, chipping away every month. The bankers figured out it’s cheaper to save the existing customers rather than to attract new ones.
We need to predict whether a customer will leave the bank soon. You have the data on clients’ past behavior and termination of contracts with the bank. 

Build a model with the maximum possible F1 score. To pass the project, you need an F1 score of at least 0.59. Check the F1 for the test set. Additionally, measure the AUC-ROC metric and compare it with the F1.
Data source: https://code.s3.yandex.net/datasets/Churn.csv

##### Business Statement

One of the key business metrics (along with cash flow etc.) for banks, internet providers, pay TV companies, telecom companies is customer attrition analysis (or customer churn analysis). We say customer churn is loss of clients or customers. It is cheaper to keep existing customers than go for new ones. Beta bank have already seen the effect of customer churn as it affects their end of the year revenue and monthly recurring revenue (or MRR). To this end, we need to predict whether a customer will leave a bank soon given their past relationship and behavior while operating with Beta bank. The bank hopes to deploy churn prediction models and effective retention strategies in managing customer attrition thereby preventing significant loss of revenue from defecting customers.

##### Task Statement

Using the customer data, train a model that predicts whether a customer will leave the bank soon.

## Data description

The data can be found in '/datasets/Churn.csv' file. Download the dataset. 

**Features**

 - `RowNumber` — data string index
 - `CustomerId` — unique customer identifier
 - `Surname` — surname
 - `CreditScore` — credit score
 - `Geography` — country of residence
 - `Gender` — gender
 - `Age` — age
 - `Tenure` — period of maturation for a customer’s fixed deposit (years)
 - `Balance` — account balance
 - `NumOfProducts` — number of banking products used by the customer
 - `HasCrCard` — customer has a credit card
 - `IsActiveMember` — customer’s activeness
 - `EstimatedSalary` — estimated salary

**Target**

 - `Exited` — сustomer has left

## Objectives

The objectives of this project is to:
- Develop a model that would predicts whether a customer will leave the bank soon
- Build a machine learning model with the maximum possible F1 score of atleast 0.59 or higher.
- Measure the AUC-ROC metric and compare it with the F1 score.

<hr>

 # Table of contents

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#open_the_data">Open the data file and study the general information</a></li>
        <li><a href="#data_preparation">Prepare the data</a></li>
        <li><a href="#feature_engineering">Feature engineering</a></li>
        <li><a href="#class_balance">Examine the balance of classes</a></li>
        <li><a href="#investigate_models">Improve the quality of the model</a></li>
        <li><a href="#check_quality">Check model quality</a></li>
        <li><a href="#sanity_check">Sanity check the model</a></li>
        <li><a href="#overall_conclusion">Overall conclusion</a></li>
    </ol>
</div>
<br>
<hr>

<div id="open_the_data">
    <h2>Open the data file and study the general information</h2> 
</div>

We require the following libraries: *pandas* and *numpy* for data preprocessing and manipulation, *Scikit-Learn* for building our learning algorithms

In [1]:
# import pandas and numpy for data preprocessing and manipulation
import numpy as np
import pandas as pd
import random

# matplotlib for visualization
import matplotlib.pyplot as plt
%matplotlib inline

# import train_test_split to split data
from sklearn.model_selection import train_test_split

# import machine learning module from the sklearn library
#from sklearn.tree import DecisionTreeClassifier # import decision tree classifier
#from sklearn.linear_model import LogisticRegression # import logistic regression 
#from sklearn.ensemble import RandomForestClassifier # import random forest algorithm
#from sklearn.ensemble import AdaBoostClassifier # import adaboost classifier algorithm

# import metrics for sanity check on model
#from sklearn.metrics import accuracy_score
#from sklearn.metrics import confusion_matrix, classification_report
#from sklearn.metrics import precision_score, recall_score, f1_score
#from sklearn.metrics import balanced_accuracy_score, roc_auc_score

print('Project libraries has been successfully been imported!')

Project libraries has been successfully been imported!


In [2]:
# read the data
try:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/Churn.csv')
except:
    df = pd.read_csv('C:/Users/hotty/Desktop/Practicum by Yandex/Projects/Supervised Learning/Churn.csv')
print('Data has been read correctly!')

Data has been read correctly!


In [3]:
# function to determine if columns in file have null values
def get_percent_of_na(df, num):
    count = 0
    df = df.copy()
    s = (df.isna().sum() / df.shape[0])
    for column, percent in zip(s.index, s.values):
        num_of_nulls = df[column].isna().sum()
        if num_of_nulls == 0:
            continue
        else:
            count += 1
        print('Column {} has {:.{}%} percent of Nulls, and {} of nulls'.format(column, percent, num, num_of_nulls))
    if count != 0:
        print("\033[1m" + 'There are {} columns with NA.'.format(count) + "\033[0m")
    else:
        print()
        print("\033[1m" + 'There are no columns with NA.' + "\033[0m")
        
# function to display general information about the dataset
def get_info(df):
    """
    This function uses the head(), info(), describe(), shape() and duplicated() 
    methods to display the general information about the dataset.
    """
    print("\033[1m" + '-'*100 + "\033[0m")
    print('Head:')
    print()
    display(df.head())
    print('-'*100)
    print('Info:')
    print()
    display(df.info())
    print('-'*100)
    print('Describe:')
    print()
    display(df.describe())
    print('-'*100)
    display(df.describe(include='object'))
    print()
    print('Columns with nulls:')
    display(get_percent_of_na(df, 4))  # check this out
    print('-'*100)
    print('Shape:')
    print(df.shape)
    print('-'*100)
    print('Duplicated:')
    print("\033[1m" + 'We have {} duplicated rows.\n'.format(df.duplicated().sum()) + "\033[0m")
    print()

In [4]:
# study the general information about the dataset 
print('General information about the dataframe')
get_info(df)

General information about the dataframe
----------------------------------------------------------------------------------------------------
Head:



,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


----------------------------------------------------------------------------------------------------
Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory u

None

----------------------------------------------------------------------------------------------------
Describe:



...

----------------------------------------------------------------------------------------------------


,Surname,Geography,Gender
count,10000,10000,10000
unique,2932,3,2
top,Smith,France,Male
freq,32,5014,5457



Columns with nulls:
Column Tenure has 9.0900% percent of Nulls, and 909 of nulls
There are 1 columns with NA.


None

----------------------------------------------------------------------------------------------------
Shape:
(10000, 14)
----------------------------------------------------------------------------------------------------
Duplicated:
We have 0 duplicated rows.




**Conclusion**

From the information about the dataset, we have 10000 rows and 14 features. Looking at the dataset, we can see that about 9% of the data is missing in the `Tenure` column. We should also note that the missing values are *missing at random (MAR)*. To handle this missing values, we could either drop them entirely since the percentage of missing values is less than 10% or replace by the median of the column. Also, we need to correct the datatype from float to int in the `Tenure`, `Balance` and `EstimatedSalary` columns.  

<div id="data_preparation">
    <h2>Prepare the data</h2> 
</div>

#### Processing Missing Values

##### Prepare `Tenure` column

To replace missing values in the `Tenure` column, we first get the unique values of `Surname`, then get the list of possible `Tenure` for those names. We then choose a random value from the list (excluding the nan values) and assign that to the missing tenure for that surname in the dataframe. For unique surname with an empty list, we use the median of the value in the `Tenure` column.

In [5]:
# replace missing values in the Tenure column
# get unique values of name from this dataframe
for surname in df['Surname'].unique().tolist():
    # get specific 'Surname' possible Tenure
    specific_surname_df = df[df['Surname'] == surname].dropna()['Tenure']
    surname_tenure_list = specific_surname_df.unique().tolist()
    # for the missing values, assign a random choice of the tenure for that surname. The default is the median of the 'Tenure'
    if surname_tenure_list != []:
        df.loc[(df['Surname'] == surname) & (df['Tenure'] != df['Tenure']), 'Tenure'] = random.choice(surname_tenure_list)
    else:
        df.loc[(df['Surname'] == surname) & (df['Tenure'] != df['Tenure']), 'Tenure'] = df['Tenure'].median()

We have replaced missing values in the `Tenure` column based on the condition we specified. Let's look at the statistics of the dataframe.

In [6]:
# check the statistics of the new dataframe
df.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.020500,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.874318,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [7]:
# re-check for missing values
df.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [8]:
# convert data to the correct data type
def convert_to_type(df, cols, type_val):
    for col in cols:
        df[col] = df[col].astype(type_val)
        
convert_to_type(df, ['Surname', 'Geography', 'Gender'], str)
convert_to_type(df, ['RowNumber', 'CustomerId', 'CreditScore', 'Age', 'Tenure', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'Exited'], 'int64')
convert_to_type(df, ['Balance', 'EstimatedSalary'], float)

In [9]:
# information about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


Now we don't have any missing values and we have changed the datatype in the dataset. Replacing the missing values seems like a better option than dropping the columns with missing values. Care should be taken when replacing missing values. We don't want to create bias or variance in our dataset. The data has been cleaned and so it is ready for feature engineering and machine learning. 

<div id="feature_engineering">
    <h2>Feature engineering</h2> 
</div>

<div id="class_balance">
    <h2>Examine the balance of classes</h2> 
</div>

<div id="investigate_models">
    <h2>Improve the quality of the model</h2> 
</div>

<div id="check_quality">
    <h2>Check model quality</h2> 
</div>

<div id="sanity_check">
    <h2>Sanity check the model</h2> 
</div>

<div id="overall_conclusion">
    <h2>Overall conclusion</h2> 
</div>